> # Long sequence

Train with long sequence can have long time and be unstable. And model can forget some steps at first.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

- Unstable gradient

We can use some methods in RNN that we used to solve gradient problem in DNN. But activaiton function that doesn't converge is not a good idea. Output and gradient can explode. So we can use tanh activation function. And gradient clipping also can be good solution to prevent gradient exploding.

RNN can't effectively use batch normalization method. We can use it by adding batch normalization layer atmemory cell. But it doesn't make better score. Another method that fit better is layer normalization. This normalize about feature dimension. One advantage is that it can independently calculate statistics of each time step. This means it operates same both train and test. Layer noramlization train one scale and moving parameter at each input. It is used after linear combination of input and hidden states.

To use it in RNN, we have to define user memory cell.

In [4]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwarg):
        super().__init__(**kwarg)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = keras.layers.LayerNormalization()
        self.activation = keras.activations.get(activation)
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [5]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])